#1.1. Charge un fichier joblib contenant une régression linéaire entraînée

In [6]:
import joblib
import numpy as np
import pandas as pd
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd /content/drive/MyDrive/ST2IASE/transpilers/C
%ls

/content/drive/MyDrive/ST2IASE/transpilers/C
linear_model.py  logistic_model.py  run_script.ipynb  transpiler.ipynb


In [9]:
dir_path = "/content/drive/MyDrive/ST2IASE/transpilers/C/"

In [10]:
!python /content/drive/MyDrive/ST2IASE/transpilers/C/linear_model.py

In [11]:
trained_model = joblib.load(dir_path + "linear.joblib")

#1.2. Récupère les valeurs de coefficients 


In [12]:
coef = trained_model.coef_
intercept = trained_model.intercept_

#1.3. Génère une chaîne de caractère contenant le code C permettant de calculer la prédiction du modèle (float prediction(float *features, int n_feature) )avec les valeur du coefficient

In [13]:
code = f"""
#include <stdio.h>

float prediction(float *features, int n_features) {{
  float pred = {intercept};
"""

n_features = len(coef)
for i in range(n_features):
    code += f"  pred += {coef[i]} * features[{i}];\n"

code += """
  return pred;
}
"""

#1.4. Génère une fonction main qui permet d'appeler prediction sur une donnée définié par un tableau statique de votre choix.

In [14]:
code += """
int main() {
  float features_test[3] = {205.0,2.0,0.0};
  float pred = prediction(features_test,3);
  printf("Predicted price is %f",pred);
  return 0;
}
"""

#1.5. Sauvegarde le code c généré dans un fichier.c 

In [15]:
with open('linear.c', 'w') as f:
    f.write(code)

#1.6. Et affiche la commande de compilation à lancer pour le compiler ou le compile directement.

In [16]:
cmd = "gcc -W -o linear linear.c"
print("Compilation linear:",cmd)

Compilation linear: gcc -W -o linear linear.c


#4. Modifier votre fichier transpileur afin qu'il permette désormais de transpiler des régression logistiques

In [17]:
!python /content/drive/MyDrive/ST2IASE/transpilers/C/logistic_model.py

In [18]:
trained_model = joblib.load(dir_path + 'logistic.joblib')

In [19]:
intercept = trained_model.intercept_
coef = trained_model.coef_[0]

In [20]:
code1 = f"""
#include <stdio.h>

float linear_regression_prediction(float *features) {{
  float pred = {intercept};
"""

for i in range(len(coef)):
  code1+= f"  pred += features[i]*{coef[i]};\n"

code1+= """
  return pred;

float exp_approx(float x, int n_term) {{
  float res = 1;
  float term = 1;
  for (int i=1; i<n_term+1;i++) {{
    term *= x/i;
    res += term;
  }}
      
  return res;
}}

float sigmoid(float x) {{
  return 1/(1 + exp_approx(-x,10));
}}

float  logistic_regression(float *features) {{
  float z = linear_regression_prediction(float *features);
  return sigmoid(z);
}}
"""

In [21]:
code1 += f"""
int main() {{
  float features[3] = A DEFINIR;
  logistic_regression(features,{intercept},{coef})
}}
"""

In [22]:
with open('logistic.c', 'w') as f:
    f.write(code1)

In [23]:
cmd = "gcc -W -o logistic logistic.c"
print("Compilation logistic:",cmd)

Compilation logistic: gcc -W -o logistic logistic.c


#5. Modifier votre fichier transpileur afin qu'il permette désormais de transpiler des arbres de décisions

In [24]:
code3 = f"""int simple_tree(float *features, float *th) {{
    return !(features[0]>th[0]) & !(features[1]>th[1]);
}}
"""